In [1]:
import pylcp
import numpy as np
from matplotlib import pyplot as plt
from scipy import constants as const

In [2]:
atom = pylcp.atom('87Rb')

In [135]:
gamma = 1/atom.state[2].tau # Hz
k = 1e2*2*np.pi*atom.transition[1].k # m^-1
#t_unit = 2*np.pi/gamma
t_unit = 1e-6
m_unit = 2*np.pi/k
velocity_unit = m_unit/t_unit
accel_unit = m_unit/t_unit**2
Hz_unit = 1/t_unit
Js_unit = const.hbar # kg m^2/s
mass_unit = Js_unit*t_unit/m_unit**2
HzperT_unit = const.value("Bohr magneton")/(Js_unit)
T_unit = Hz_unit/HzperT_unit
amu_unit = mass_unit/1.66e-27
cm_unit = m_unit/1e-2
F_unit = mass_unit*m_unit/t_unit**2

In [136]:
ksim=k*m_unit
gammasim=gamma/Hz_unit

In [137]:
Hg, Bgq = pylcp.hamiltonians.singleF(F=0, gF=0, muB=1e-4*cm_unit*HzperT_unit/Hz_unit)
He, Beq = pylcp.hamiltonians.singleF(F=1, gF=1, muB=1e-4*cm_unit*HzperT_unit/Hz_unit)
dijq = pylcp.hamiltonians.dqij_two_bare_hyperfine(0, 1)
ham = pylcp.hamiltonian(Hg, He, Bgq, Beq, dijq, mass=85.4678/amu_unit,k=0, gamma=gammasim)
mag_field = pylcp.fields.quadrupoleMagneticField(15) # G/cm!! axial

In [142]:
H_g_D2, mu_q_g_D2 = pylcp.hamiltonians.hyperfine_coupled(
    atom.state[0].J, atom.I, atom.state[0].gJ, atom.gI,
    atom.state[0].Ahfs/Hz_unit, Bhfs=0, Chfs=0,
    muB=1e-4*cm_unit*HzperT_unit/Hz_unit)
H_e_D2, mu_q_e_D2 = pylcp.hamiltonians.hyperfine_coupled(
    atom.state[2].J, atom.I, atom.state[2].gJ, atom.gI,
    Ahfs=atom.state[2].Ahfs/Hz_unit,
    Bhfs=atom.state[2].Bhfs/Hz_unit, Chfs=0,
    muB=1e-4*cm_unit*HzperT_unit/Hz_unit)

dijq_D2 = pylcp.hamiltonians.dqij_two_hyperfine_manifolds(
    atom.state[0].J, atom.state[2].J, atom.I)

E_e_D2 = np.unique(np.diagonal(H_e_D2))
E_g_D2 = np.unique(np.diagonal(H_g_D2))

hamiltonian_D2 = pylcp.hamiltonian(H_g_D2, H_e_D2, mu_q_g_D2, mu_q_e_D2, dijq_D2, mass=85.4678/amu_unit,k=0, gamma=gammasim)

In [161]:
s=1
wb = 1/cm_unit
rstop = 1/cm_unit

def MOT_Beams(det_MOT, *args):
    return pylcp.laserBeams([
        {'kvec':np.array([-1/np.sqrt(2), -1/np.sqrt(2), 0.])*ksim, 'pol':-1, 'delta':(det_MOT)/Hz_unit, 's': s,'wb':wb,'rs':rstop},
        {'kvec':np.array([1/np.sqrt(2), 1/np.sqrt(2), 0.])*ksim, 'pol':-1, 'delta':(det_MOT)/Hz_unit, 's': s,'wb':wb,'rs':rstop},
        {'kvec':np.array([1/np.sqrt(2), -1/np.sqrt(2), 0.])*ksim, 'pol':-1, 'delta':(det_MOT)/Hz_unit, 's': s,'wb':wb,'rs':rstop},
        {'kvec':np.array([-1/np.sqrt(2), 1/np.sqrt(2), 0.])*ksim, 'pol':-1, 'delta':(det_MOT)/Hz_unit, 's': s,'wb':wb,'rs':rstop},
        {'kvec':np.array([0., 0.,  1.])*ksim, 'pol':+1, 'delta':(det_MOT)/Hz_unit, 's': s,'wb':wb,'rs':rstop},
        {'kvec':np.array([0., 0., -1.])*ksim, 'pol':+1, 'delta':(det_MOT)/Hz_unit, 's': s,'wb':wb,'rs':rstop}
    ], beam_type=pylcp.clippedGaussianBeam)
    
laserBeams_cooling_D2 = pylcp.conventional3DMOTBeams(
    s=s, delta=(E_e_D2[-1] - E_g_D2[-1]) + -1*gammasim, wb = wb, rs= rstop, k = ksim, beam_type=pylcp.clippedGaussianBeam,rotation_angles=[np.pi/4,0,0])
laserBeams_repump_D2 = pylcp.conventional3DMOTBeams(
    s=0.01*s, delta=(E_e_D2[-2] - E_g_D2[-2]), wb = wb, rs= rstop, k = ksim, beam_type=pylcp.clippedGaussianBeam,rotation_angles=[np.pi/4,0,0])
laserBeams_D2 = laserBeams_cooling_D2 + laserBeams_repump_D2

In [162]:
rateeq = pylcp.rateeq(laserBeams_D2, mag_field, hamiltonian_D2,include_mag_forces=False)

r = np.linspace(-1/cm_unit, 1/cm_unit, 100)
v = np.linspace(-20/velocity_unit, 20./velocity_unit, 100)

R, V = np.meshgrid(r, v)
rateeq.generate_force_profile([R, np.zeros(R.shape), np.zeros(R.shape)],
                           [V, np.zeros(V.shape), np.zeros(V.shape)],
                           name='Frad', progress_bar=True)

angle = float(0)
vel = float(5)

rateeq.set_initial_position_and_velocity(np.array([-1/cm_unit, 0., 0.]),np.array([vel*np.cos(np.radians(angle))/velocity_unit, vel*np.sin(np.radians(angle))/velocity_unit, 0.]))
rateeq.set_initial_pop_from_equilibrium()
rateeq.evolve_motion([0., 5e-2/t_unit], progress_bar=True, max_step = 10e-5/t_unit, rtol=1e-3, atol=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1/velocity_unit,1/velocity_unit,1/velocity_unit,1e-3/cm_unit,1e-3/cm_unit,1e-3/cm_unit],method="LSODA")
sol = rateeq.sol

fig, ax = plt.subplots(1, 1)
ax.plot(sol.r[0]*cm_unit,sol.v[0]*velocity_unit,'b-')
colormesh = ax.pcolormesh(R*cm_unit, V*velocity_unit, rateeq.profile['Frad'].F[0]*F_unit*1e-3/(112*1.66e-27), cmap = 'viridis')
cb1 = plt.colorbar(colormesh)
cb1.set_label('$a (km/s^2)$')
ax.set_xlabel('$x$ (cm)')
ax.set_ylabel('$v(m/s)$')
fig.subplots_adjust(left=0.12,right=0.9)
ax.set_xlim([-10,10])
ax.set_ylim([-20,20])

Completed in 22.20 s.                                               


ValueError: Enountered a NaN!